# **DESCRIPTA.AI** -> Extension Tables AppScript Code: (Code.gs)

This script implements the client in Google Apps Script, allowing the user, from a Google Sheets spreadsheet, to upload a .zip file, send it to a Flask server via HTTP (ngrok), and automatically process the results. If the processed file includes .xlsx spreadsheets, they are directly imported into the active document.

* PIPELINE_FOLDER_NAME

Constant that defines the name of the root folder in Google Drive where both the original ZIP files and the results generated by the backend will be stored.

* getDestinationFolder()


Function that searches for or creates the destination folder within the user's Drive. It ensures that all files operate from a controlled location.

* onOpen()

Function that runs automatically when the Google Sheets file is opened. It adds a custom menu called *Pipeline ZIP* to the user interface, with an option to upload and process ZIP files.


* showSidebar()

Displays a sidebar with a custom HTML interface (from the file *Sidebar.html*, not included here), which allows the user to select a ZIP file for processing.



* uploadAndProcess(b64data, filename)

Main function that:

Decodes the received ZIP file from base64.

Saves it as a file in Drive.

Calls processAndSaveResults() to process it via Flask.

If the output ZIP contains .xlsx files, it automatically imports them into the active sheet using importExcelToActiveSpreadsheet.

* processAndSaveResults(driveFileId, folder)

Sends the ZIP file to the /run_pipeline endpoint of the Flask server using multipart/form-data.

The expected result is a ZIP file containing multiple files (JSON, DOCX, XLSX).

It unzips the received file and saves each file to the specified folder.

Returns a list of objects {name, url, id} for each extracted file.

* importExcelToActiveSpreadsheet(fileId)

Imports a .xlsx file into the active Google Sheets document:

Converts the file to Google Sheets format.

Copies each sheet into the current document.

Deletes the original sheets to avoid errors such as “a spreadsheet must have at least one sheet.”

Moves the temporary sheet to the trash to keep the workspace clean.


* testProcess()

Manual helper function for testing from the Apps Script editor. It processes a previously uploaded ZIP file (by ID) and logs the results.


In [ ]:
/**
 * Code.gs
 * ----------------------------------------------------
 * Controlador de Apps Script para subir un ZIP local,
 * enviarlo a tu servidor Flask (vía ngrok), guardar
 * resultados en carpeta y, si hay un .xlsx, importarlo
 * directamente en la hoja activa SIN provocar errores
 * al eliminar todas las pestañas.
 */

/** Nombre de la carpeta raíz donde centralizamos todo */
const PIPELINE_FOLDER_NAME = 'Pipeline ZIP';

/**
 * Busca o crea la carpeta de destino en la raíz de Drive
 */
function getDestinationFolder() {
  const root = DriveApp.getRootFolder();
  const it = root.getFoldersByName(PIPELINE_FOLDER_NAME);
  return it.hasNext() ? it.next() : root.createFolder(PIPELINE_FOLDER_NAME);
}

/** 1) Menú al abrir la hoja */
function onOpen() {
  SpreadsheetApp.getUi()
    .createMenu('Pipeline ZIP')
    .addItem('Cargar & Procesar ZIP', 'showSidebar')
    .addToUi();
}

/** 2) Sidebar para selector de archivos */
function showSidebar() {
  const html = HtmlService
    .createHtmlOutputFromFile('Sidebar')
    .setTitle('Subir y procesar ZIP');
  SpreadsheetApp.getUi().showSidebar(html);
}

/**
 * 3) Recibe ZIP en Base64, lo guarda en carpeta y procesa.
 *    Luego importa a la hoja activa cada .xlsx que encuentre.
 */
function uploadAndProcess(b64data, filename) {
  // 3.1) Crear blob ZIP
  const bytes = Utilities.base64Decode(b64data);
  const zipBlob = Utilities.newBlob(bytes, 'application/zip', filename);

  // 3.2) Guardar ZIP en carpeta
  const folder = getDestinationFolder();
  const zipFile = folder.createFile(zipBlob);

  // 3.3) Procesar remoto y guardar resultados
  const results = processAndSaveResults(zipFile.getId(), folder);

  // 3.4) Para cada resultado .xlsx, importarlo
  results.forEach(r => {
    if (/\.xlsx$/i.test(r.name)) {
      importExcelToActiveSpreadsheet(r.id);
    }
  });

  return results;
}

/**
 * 4) Llama al endpoint, descomprime y guarda blobs en carpeta,
 *    devolviendo {name, url, id} de cada archivo.
 */
function processAndSaveResults(driveFileId, folder) {
  const zipBlob = DriveApp.getFileById(driveFileId).getBlob();
  const ngrokUrl = 'https://2417-190-96-108-30.ngrok-free.app/run_pipeline';

  // ENVÍO multipart/form-data en lugar de application/zip
  const formData = {
    file: zipBlob
  };
  const resp = UrlFetchApp.fetch(ngrokUrl, {
    method: 'post',
    payload: formData,         // Apps Script detecta multipart automáticamente
    muteHttpExceptions: true,
    followRedirects: true
  });
  const code = resp.getResponseCode();
  const text = resp.getContentText().slice(0, 300);
  Logger.log(HTTP ${code} → ${text});
  if (code < 200 || code > 299) {
    throw new Error(Error pipeline (${code}):\n${text});
  }

  const resultZip = resp.getBlob().setName('resultados.zip');
  const blobs = Utilities.unzip(resultZip);
  return blobs.map(blob => {
    const file = folder.createFile(blob.setName(blob.getName()));
    return { name: file.getName(), url: file.getUrl(), id: file.getId() };
  });
}


/**
 * 5) Importa *tal cual* un .xlsx (por ID) en la hoja activa,
 *    convirtiéndolo a Google Sheets y reemplazando solo las
 *    pestañas originales, para evitar el error de “todas las hojas”.
 */
function importExcelToActiveSpreadsheet(fileId) {
  // 5.1) Copiar+convertir .xlsx a Google Sheets temporal
  const fileMeta = {
    name: 'TMP_IMPORT_' + Date.now(),
    mimeType: MimeType.GOOGLE_SHEETS
  };
  const tempFile = Drive.Files.copy(fileMeta, fileId);
  const tempId   = tempFile.id;

  // 5.2) Abrir temporal y activo
  const tempSS   = SpreadsheetApp.openById(tempId);
  const activeSS = SpreadsheetApp.getActiveSpreadsheet();

  // 5.3) Registrar IDs de las hojas que ya estaban
  const originalIds = activeSS.getSheets().map(s => s.getSheetId());

  // 5.4) Copiar cada hoja del temporal al activo
  tempSS.getSheets().forEach(s => {
    const copied = s.copyTo(activeSS);
    copied.setName(s.getName());
  });

  // 5.5) Eliminar únicamente las pestañas originales
  originalIds.forEach(id => {
    const sheet = activeSS.getSheetById(id);
    if (sheet) activeSS.deleteSheet(sheet);
  });

  // 5.6) Enviar el Google Sheet temporal a la papelera
  DriveApp.getFileById(tempId).setTrashed(true);
}

/** 6) Test manual desde editor */
function testProcess() {
  const originalZipId = 'TU_ZIP_FILE_ID';
  const folder = getDestinationFolder();
  const results = processAndSaveResults(originalZipId, folder);
  results.forEach(r => Logger.log(${r.name}: ${r.url}));
}
